In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [11]:
vine_df = pd.read_csv("vine_data.csv")
vine_df.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
0,RTIS3L2M1F5SM,5,0,0,N,Y
1,R1ZV7R40OLHKD,5,0,0,N,Y
2,R3BH071QLH8QMC,1,0,1,N,Y
3,R127K9NTSXA2YH,3,0,0,N,Y
4,R32ZWUXDJPW27Q,4,0,0,N,Y


In [12]:
vine_total_votes_df = vine_df.loc[vine_df["total_votes"]>= 20]
vine_total_votes_df.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
55,R4PKAZRQJJX14,1,21,34,N,N
74,R2CI0Y288CC7E2,1,21,35,N,Y
209,R127WEQY2FM1T3,1,147,175,N,Y
289,R3EZ0EPYLDA34S,1,14,31,N,Y
483,R2FJ94555FZH32,2,55,60,N,N


In [13]:
helpful_votes_df = vine_total_votes_df.loc[vine_total_votes_df["helpful_votes"]/vine_total_votes_df["total_votes"]*100>=50]
helpful_votes_df.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
55,R4PKAZRQJJX14,1,21,34,N,N
74,R2CI0Y288CC7E2,1,21,35,N,Y
209,R127WEQY2FM1T3,1,147,175,N,Y
483,R2FJ94555FZH32,2,55,60,N,N
537,R1U3AR67RE273L,1,51,65,N,Y


In [14]:
vine_yes_df = helpful_votes_df.loc[helpful_votes_df["vine"]== 'Y']
vine_yes_df.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
32611,R3KKUSGFZWSUIY,5,56,63,Y,N
33112,R10FO5UKKVZBK2,3,23,23,Y,N
69680,RM4KSGEOR7MU1,5,19,24,Y,N
155361,RG7VRMYLEXD23,4,22,26,Y,N
239327,R11O4YSCPSNL6L,3,20,26,Y,N


In [15]:
vine_no_df = helpful_votes_df.loc[helpful_votes_df["vine"]== 'N']
vine_no_df.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
55,R4PKAZRQJJX14,1,21,34,N,N
74,R2CI0Y288CC7E2,1,21,35,N,Y
209,R127WEQY2FM1T3,1,147,175,N,Y
483,R2FJ94555FZH32,2,55,60,N,N
537,R1U3AR67RE273L,1,51,65,N,Y


In [16]:
vine_paid_df = vine_yes_df.loc[vine_yes_df["star_rating"] == 5]
vine_paid_df.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
32611,R3KKUSGFZWSUIY,5,56,63,Y,N
69680,RM4KSGEOR7MU1,5,19,24,Y,N
335897,R286MFBAJ8NPD6,5,46,51,Y,N
364816,R1JRR530H4COA2,5,22,28,Y,N
367440,RQ5WD90PUNBU9,5,21,24,Y,N


In [17]:
vine_paid_df.count()

review_id            48
star_rating          48
helpful_votes        48
total_votes          48
vine                 48
verified_purchase    48
dtype: int64

In [18]:
vine_unpaid_df = vine_no_df.loc[vine_no_df["star_rating"]== 5]
vine_unpaid_df.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
1402,R36O341WWXXKNP,5,28,31,N,N
2507,R29BOS5HMAY1LO,5,88,110,N,N
3932,RDX2ZZ46AM343,5,32,34,N,Y
4133,R60GI4Z1CNGGV,5,17,20,N,Y
5278,R2FAARI3JQO9XQ,5,29,30,N,Y


In [19]:
vine_unpaid_df.count()

review_id            15663
star_rating          15663
helpful_votes        15663
total_votes          15663
vine                 15663
verified_purchase    15663
dtype: int64